# Lecture 2 Transactions

## Exercise 1: Digital signatures

Below are a few example functions from the `ecdsa` package.
* Try to create and verify a signature for "hello world!". 
* Check that verification fails if you change the message.
* Complete the verify function that recreates the public key, and verifies the signature inside a try escape clause, and returns **True** or **False**.
* Optional: Can you write functions to convert the public key to hexadecimals, and back?

In [43]:
from ecdsa import SigningKey, VerifyingKey

# generate random secret key
sk = SigningKey.generate() # uses NIST192p
# get matching public key
vk = sk.verifying_key
# sign a messge
signature = sk.sign(b"message")
# verify message and print result
print(vk.verify(signature, b"message"))
# transfer public key into bytes string
vkeystring = vk.to_string()
# get back public key from bytes string
vk2 = VerifyingKey.from_string(vkeystring)
# verify again
print(vk2.verify(signature, b"message"))

True
True


In [44]:
def verify(vkeystring, signature, messagestring):
    # convert string to public key and verify signature
    # use try except to catch all errors and return true or false
    try:
        vk = VerifyingKey.from_string(vkeystring)
        vk.verify(signature, messagestring.encode())
        print("verify success!")
        return True
    except:
        print("Could not verify signature")
    return False

In [45]:
verify(vkeystring,signature,"message")

verify success!


True

## Exercise 2: Accounts

Below is a simple bak with account balances.
* Check the pseudocode.
* Run the examle below.
* Update the bank with a counter, so the transfer is no longer vulnerable to reply.

In [51]:
class Bank:
    def __init__(self):
        # A dictionary of accounts
        self.accounts = {}
        self.counter = {}
    def deposit(self,key, value):
        assert (value > 0)
        if self.accounts.get(key,0) != 0:
            self.accounts[key] += value
        else:
            self.accounts[key] = value
    
    def transfer(self, fromk, tok, value, cnt, sig):
        assert (value > 0)
        assert (self.accounts.get(fromk,0) >= value)
        assert (verify(fromk,sig,str(tok) + str(value)+ str(cnt)))
        assert (cnt == self.counter.get(fromk,0))
        self.accounts[fromk] -= value
        self.counter[fromk] = self.counter.get(fromk,0)+1
        if self.accounts.get(tok,0) != 0:
            self.accounts[tok] += value
        else:
            self.accounts[tok] = value
        if self.accounts[fromk] == 0:
            del self.accounts[fromk]

In [55]:
# create to secret and public keys
sk1 = SigningKey.generate()
sk2 = SigningKey.generate()
vk1s = sk1.verifying_key.to_string()
vk2s = sk2.verifying_key.to_string()


# create bank and deposit
bank = Bank()
bank.deposit(vk1s, 100)
bank.deposit(vk2s, 100)

# create signature this depends on you you implement verification
sig = sk1.sign((str(vk2s) + str(25) + str(0)).encode())

# transfer money
bank.transfer(vk1s, vk2s, 25,0, sig)

# check new balances
print("vk1 has balance {}".format(bank.accounts[vk1s]))
print("vk2 has balance {}".format(bank.accounts[vk2s]))

# after the transfer above goes through, it should not be possible to replay!
try:
    bank.transfer(vk1s, vk2s, 25,0, sig)
    print("Vulnerable to replay attack!")
except:
    print("Not vulnerable to replay!")

verify success!
vk1 has balance 75
vk2 has balance 125
Not vulnerable to replay!


In [48]:
print("vk1 has balance {}".format(bank.accounts[vk1s]))
print("vk2 has balance {}".format(bank.accounts[vk2s]))


vk1 has balance 50
vk2 has balance 150


## Exercise 3: UTXO
Below is a stub for an UTXO based bank.
The bank contains:
* `utxo`: a dictionary with all unspent outputs.
* `counter`: a counter used to create new ids for new outputs.

An **output** is a touple `(value, key)` where
* `value` is a number (the amount of money the output is worth.
* `key` is a verifying (public) key that gives the owner of the output.

The bank has methods
* `deposit` to create a new output in the `utxo`, 
* `transferOne` to transfer the value from one output and 
* `transfer` to transfer the value from multiple outputs.

### transfer one output
Implement `transferOne` method that allows to transfer the value from one output to someone else.
    The function has arguments:
    * `input` a `(id, sig)` tuple. 
        - `id` is the key in the utxo map (counter value) 
        - `sig` is a signature with the public key stored in `utxo[id]`
    * `output` a `(value, key)` tuple like in `deposit`
    
   You need to check the following:
    * `utxo[id]` exists
    * `sig` is correct. The message that should be signed must contain both `id` and `output`.
    * `output[0]` the `value` is less or equal than the `value` in `utxo[id]`.
        
   If that is the case remove `id` from `utxo` and add `output` to `utxo`.
   
### transfer multiple outputs   
Implement `transfer` which takes multipe `inputs` and creates multiple `outputs`.
   Note that the different inputs must have a signature each.
   All `id`s and `outputs` should be part of the signature.

In [33]:
class UTXOBank:
    def __init__(self):
        self.utxo ={}
        self.counter = 0
    def deposit(self, key, value):
        self.utxo[self.counter]= (value,key)
        self.counter += 1
        
    def transferOne(self, input, output):
        """This function takes one input and returns one output
            * Input is a tuple (id, sig), where
                * id is a key (counter) of the utxo dictionary
                * sig is a signature with the key given in that dictionary
            * Output is a tuple (value, key)
            * The value of the new output must be less or equal to the value of the input
            * Used output is removed from UTXO, new output is added.
        """
        pass
        
    def transfer(self,inputs, outputs):
        # inputs is a list of tuples (id, sig)
        # outputs is a list of tupbles (value, key)
        pass

In [42]:
# create to secret and public keys
sk1 = SigningKey.generate()
sk2 = SigningKey.generate()
vk1s = sk1.verifying_key.to_string()
vk2s = sk2.verifying_key.to_string()


# create bank and deposit
bank = UTXOBank()
bank.deposit(vk1s, 100)
bank.deposit(vk2s, 100)

# create signature this depends on you you implement verification
sig = sk1.sign((str(0) + str((100,vk2s))).encode())

bank.transferOne((0,sig),(100,vk2s))
#bank.transferOne((0,sig),(100,vk2s))

for cnt, output in bank.utxo.items():
    print(cnt, "({},{})".format(output[0], output[1][:16]))


verify success!
verify success!
1 (100,b'\xe1\xdc\xfc\xe0\xfb\xf9\x19\x83\x86\x83\x15y\xc9\nuJ')
3 (100,b'\xc9\x96Oq:,\xdf\n&\xec\xe8\xa87\xa7\x88)')
